# Dvizenje vo site nasoki

In [144]:
from collections import deque
import heapq
from plotly import graph_objects as go
import ipywidgets as widgets
import math

In [145]:
def breadth_first_search(starting_vertex, goal_vertex):
    """
    Returns the path from starting_vertex to goal_vertex using the breadth first search algorithm.
    """
    visited = {starting_vertex}
    queue = deque([[starting_vertex]])
    while queue:
        vertex_list = queue.popleft()
        vertex_to_expand = vertex_list[-1]
        
        frontier = list(set([q[-1] for q in queue]))
        yield frontier, vertex_list, vertex_to_expand
        
        for neighbour in expand_state(vertex_to_expand):
            if neighbour not in visited:
                visited.add(neighbour)
                queue.append(vertex_list + [neighbour])
    yield [], [], goal_vertex

In [146]:
def depth_first_search(starting_vertex, goal_vertex):
    """
    Returns the path from starting_vertex to goal_vertex using the depth first search algorithm.
    """
    visited = {starting_vertex}
    queue = deque([[starting_vertex]])
    while queue:
        vertex_list = queue.popleft()
        vertex_to_expand = vertex_list[-1]
        
        frontier = list(set([q[-1] for q in queue]))
        yield frontier, vertex_list, vertex_to_expand
        
        for neighbour in expand_state(vertex_to_expand):
            if neighbour not in visited:
                visited.add(neighbour)
                queue.appendleft(vertex_list + [neighbour])
    yield [], [], goal_vertex

In [147]:
def uniform_cost_search(starting_vertex, goal_vertex):
    """
    Returns the path from starting_vertex to goal_vertex using the uniform-cost search algorithm.
    """
    expanded = set()
    queue = [(0, [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        weight, vertex_list = heapq.heappop(queue)
        vertex_to_expand = vertex_list[-1]
        global weight_end
        weight_end = weight
        if vertex_to_expand in expanded:
            continue

        frontier = list(set([q[-1][-1] for q in queue]))
        yield frontier, vertex_list, vertex_to_expand
        
        for neighbour in expand_state(vertex_to_expand):
            new_weight = get_weight(vertex_to_expand, neighbour)
            if neighbour not in expanded:
                heapq.heappush(queue, (weight + new_weight, vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)
    yield [], [], goal_vertex

In [148]:
def gready_search(starting_vertex, goal_vertex):
    """
    Returns the path from starting_vertex to goal_vertex using the gready search algorithm.
    """
    expanded = set()
    queue = [(0, [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        weight, vertex_list = heapq.heappop(queue)
        vertex_to_expand = vertex_list[-1]
        global weight_end
        weight_end = weight
        if vertex_to_expand in expanded:
            continue

        frontier = list(set([q[-1][-1] for q in queue]))
        yield frontier, vertex_list, vertex_to_expand
        
        for neighbour in expand_state(vertex_to_expand):
            heuristic = euclidean_distance(neighbour, goal_vertex)
            if neighbour not in expanded:
                heapq.heappush(queue, (heuristic, vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)
    yield [], [], goal_vertex

In [149]:
def a_star_search(starting_vertex, goal_vertex, alpha):
    """
    Returns the path from starting_vertex to goal_vertex using the a* search algorithm.
    """
    expanded = set()
    queue = [((0, 0), [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        weight_tupple, vertex_list = heapq.heappop(queue)
        current_a_star_weight, current_path_weight = weight_tupple
        vertex_to_expand = vertex_list[-1]
        global weight_end
        weight_end = current_path_weight
        if vertex_to_expand in expanded:
            continue

        frontier = list(set([q[-1][-1] for q in queue]))
        yield frontier, vertex_list, vertex_to_expand

        for neighbour in expand_state(vertex_to_expand):
            if neighbour not in expanded:
                new_weight = get_weight(vertex_to_expand, neighbour)
                heuristic = euclidean_distance(neighbour, goal_vertex)
                path_weight = current_path_weight + new_weight
                a_star_weight = path_weight + alpha * heuristic
                heapq.heappush(queue, ((a_star_weight, path_weight), vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)
    yield [], [], goal_vertex

In [150]:
def manhattan_distance(state_1, state_2):
    return abs(state_1[0] - state_2[0]) + abs(state_1[1] - state_2[1])

In [151]:
def euclidean_distance(state_1, state_2):
    return math.sqrt((state_1[0] - state_2[0])**2 + (state_1[1] - state_2[1])**2)

In [152]:
def get_weight(source_state, destination_state):
    source_x, source_y = source_state
    destination_x, destination_y = destination_state
    weight = MOUNTAIN[destination_y][destination_x][0] - MOUNTAIN[source_y][source_x][0]
    return 1 + max(weight, 0)

In [153]:
def count_score(vertex, end=False):
    global count
    text.value = str(count)
    if end:
        text.value = str(count + weight)
    count += 1

In [154]:
def expand_state(state):
    next_states = []
    x, y = state

    def is_valid(square):
        x, y = square
        if not(0 <= x < COLUMNS) or not(0 <= y < ROWS):
            return False
        for wall in WALLS:
            if wall['x'][0] < x < wall['x'][1] and wall['y'][0] < y < wall['y'][1]:
                return False
        return True

    neighbour_states = [(x+1, y), (x+1, y+1), (x, y+1), (x-1, y+1), (x-1, y), (x-1, y-1), (x, y-1), (x+1, y-1)]
    for neighbour_state in neighbour_states:
        if is_valid(neighbour_state):
            next_states.append(neighbour_state)

    return next_states

In [155]:
def update_fig(x):
    if button.disabled is True:
        return
    else:
        frontier, current_path, vertex_to_expand = next(simulation)
        count_score(vertex_to_expand)
        if vertex_to_expand == GOAL:
            button.disabled = True
            play.disabled = True
            play.value = play.max
            count_score(vertex_to_expand,True)
    path_x = [segment[0] for segment in current_path]
    path_y = [segment[1] for segment in current_path]
    frontier_x = [segment[0] for segment in frontier]
    frontier_y = [segment[1] for segment in frontier]
    with fig.batch_update():
        fig.data[1].x = path_x
        fig.data[1].y = path_y
        fig.data[2].x = frontier_x
        fig.data[2].y = frontier_y
        fig.data[3].x = [vertex_to_expand[0]]
        fig.data[3].y = [vertex_to_expand[1]]

def create_button():
    button = widgets.Button(description='Чекор')
    button.on_click(update_fig)
    return button
def create_score():
    score = widgets.Text(value='', placeholder='Score')
    return score

def create_play():
    play = widgets.Play(value=0, max=3000, interval=10, show_repeat=False)
    play.observe(update_fig, 'value')
    return play

def create_mountain():
    def f(x, y):
        return 1 - (x - 2 * COLUMNS // 3) ** 2 - (y - ROWS // 2) ** 2

    mountain = [[f(x, y) for x in range(COLUMNS)] for y in range(ROWS)]
    min_value = min(min(mountain))
    for y in range(ROWS):
        for x in range(COLUMNS):
            mountain[y][x] = [mountain[y][x] - min_value] * 3
    
    min_value = min(min(min(mountain)))
    max_value = max(max(max(mountain)))
    mountain_image = go.Image(z=mountain, zmin=[min_value]*4, zmax=[max_value]*4)
    return mountain, mountain_image

def create_fig():
    fig = go.FigureWidget()

    for wall in WALLS:
        fig.add_shape(
            type='rect', fillcolor='RoyalBlue', 
            x0=wall['x'][0], x1=wall['x'][1], 
            y0=wall['y'][0], y1=wall['y'][1])

    fig.add_trace(mountain_image)
    fig.add_scatter(mode='lines', name='Патека')
    fig.add_scatter(mode='markers', name='Фронт')
    fig.add_scatter(marker_symbol='x', marker_size=12, name='Нареден')
    fig.add_scatter(x=[START[0]], y=[START[1]], marker_symbol='x', marker_size=12, name='Почеток')
    fig.add_scatter(x=[GOAL[0]], y=[GOAL[1]], marker_symbol='x', marker_size=12, name='Цел')

    fig.update_xaxes(range=[-0.5, COLUMNS - 0.5], dtick=1)
    fig.update_yaxes(range=[-0.5, ROWS - 0.5], dtick=1)
    fig.update_layout(width=34*COLUMNS, height=36*ROWS)
    return fig

In [ ]:
ROWS, COLUMNS = 20, 30
count, weight_end = 0, 0
WALLS = [
    {'x': (12.5, 23.5), 'y': (7.5, 9.5)},
    {'x': (8.5, 10.5), 'y': (2.5, 13.5)},
    {'x': (16.5, 19.5), 'y': (12.5, 16.5)},
]

START = (4, 4)
GOAL = (29, 9)
MOUNTAIN, mountain_image = create_mountain()
simulation = a_star_search(START, GOAL, alpha=20)

play = create_play()
button = create_button()
text = create_score()
fig = create_fig()
widgets.VBox([widgets.HBox([play, button, text]), fig])

# Odbiranje na najbliskata cel

In [2]:
from collections import deque
import heapq
from plotly import graph_objects as go
import ipywidgets as widgets
import math

In [3]:
def breadth_first_search(starting_vertex, goal_vertex):
    """
    Returns the path from starting_vertex to goal_vertex using the breadth first search algorithm.
    """
    visited = {starting_vertex}
    queue = deque([[starting_vertex]])
    while queue:
        vertex_list = queue.popleft()
        vertex_to_expand = vertex_list[-1]
        
        frontier = list(set([q[-1] for q in queue]))
        yield frontier, vertex_list, vertex_to_expand
        
        for neighbour in expand_state(vertex_to_expand):
            if neighbour not in visited:
                visited.add(neighbour)
                queue.append(vertex_list + [neighbour])
    yield [], [], goal_vertex

In [4]:
def depth_first_search(starting_vertex, goal_vertex):
    """
    Returns the path from starting_vertex to goal_vertex using the depth first search algorithm.
    """
    visited = {starting_vertex}
    queue = deque([[starting_vertex]])
    while queue:
        vertex_list = queue.popleft()
        vertex_to_expand = vertex_list[-1]
        
        frontier = list(set([q[-1] for q in queue]))
        yield frontier, vertex_list, vertex_to_expand
        
        for neighbour in expand_state(vertex_to_expand):
            if neighbour not in visited:
                visited.add(neighbour)
                queue.appendleft(vertex_list + [neighbour])
    yield [], [], goal_vertex

In [5]:
def uniform_cost_search(starting_vertex, goal_vertex):
    """
    Returns the path from starting_vertex to goal_vertex using the uniform-cost search algorithm.
    """
    expanded = set()
    queue = [(0, [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        weight, vertex_list = heapq.heappop(queue)
        vertex_to_expand = vertex_list[-1]
        global weight_end
        weight_end = weight
        if vertex_to_expand in expanded:
            continue

        frontier = list(set([q[-1][-1] for q in queue]))
        yield frontier, vertex_list, vertex_to_expand
        
        for neighbour in expand_state(vertex_to_expand):
            new_weight = get_weight(vertex_to_expand, neighbour)
            if neighbour not in expanded:
                heapq.heappush(queue, (weight + new_weight, vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)
    yield [], [], goal_vertex

In [6]:
def gready_search(starting_vertex, goal_vertex):
    """
    Returns the path from starting_vertex to goal_vertex using the gready search algorithm.
    """
    expanded = set()
    queue = [(0, [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        weight, vertex_list = heapq.heappop(queue)
        vertex_to_expand = vertex_list[-1]
        global weight_end
        weight_end = weight
        if vertex_to_expand in expanded:
            continue

        frontier = list(set([q[-1][-1] for q in queue]))
        yield frontier, vertex_list, vertex_to_expand

        for neighbour in expand_state(vertex_to_expand):
            heuristic = min([euclidean_distance(neighbour, temp) for temp in goal_vertex])
            if neighbour not in expanded:
                heapq.heappush(queue, (heuristic, vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)
    yield [], [], goal_vertex

In [28]:
def a_star_search(starting_vertex, goal_vertex, alpha):
    """
    Returns the path from starting_vertex to goal_vertex using the a* search algorithm.
    """
    expanded = set()
    queue = [((0, 0), [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        weight_tupple, vertex_list = heapq.heappop(queue)
        current_a_star_weight, current_path_weight = weight_tupple
        vertex_to_expand = vertex_list[-1]
        global weight_end
        weight_end = current_path_weight
        if vertex_to_expand in expanded:
            continue

        frontier = list(set([q[-1][-1] for q in queue]))
        yield frontier, vertex_list, vertex_to_expand

        for neighbour in expand_state(vertex_to_expand):
            if neighbour not in expanded:
                new_weight = get_weight(vertex_to_expand, neighbour)
                heuristic = min([euclidean_distance(neighbour, temp) for temp in goal_vertex])
                path_weight = current_path_weight + new_weight
                a_star_weight = path_weight + alpha * heuristic
                heapq.heappush(queue, ((a_star_weight, path_weight), vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)
    yield [], [], goal_vertex

In [29]:
def manhattan_distance(state_1, state_2):
    return abs(state_1[0] - state_2[0]) + abs(state_1[1] - state_2[1])


def euclidean_distance(state_1, state_2):
    return math.sqrt((state_1[0] - state_2[0])**2 + (state_1[1] - state_2[1])**2)


def get_weight(source_state, destination_state):
    source_x, source_y = source_state
    destination_x, destination_y = destination_state
    weight = MOUNTAIN[destination_y][destination_x][0] - MOUNTAIN[source_y][source_x][0]
    return 1 + max(weight, 0)

In [30]:
def count_score(vertex, end=False):
    global count
    text.value = str(count)
    if end:
        text.value = str(count + weight_end)
    count += 1

In [31]:
def expand_state(state):
    next_states = []
    x, y = state

    def is_valid(square):
        x, y = square
        if not(0 <= x < COLUMNS) or not(0 <= y < ROWS):
            return False
        for wall in WALLS:
            if wall['x'][0] < x < wall['x'][1] and wall['y'][0] < y < wall['y'][1]:
                return False
        return True

    neighbour_states = [(x+1, y), (x+1, y+1), (x, y+1), (x-1, y+1), (x-1, y), (x-1, y-1), (x, y-1), (x+1, y-1)]
    for neighbour_state in neighbour_states:
        if is_valid(neighbour_state):
            next_states.append(neighbour_state)

    return next_states

In [32]:
def update_fig(x):
    if button.disabled is True:
        return
    else:
        frontier, current_path, vertex_to_expand = next(simulation)
        count_score(vertex_to_expand)
        if vertex_to_expand in GOALS:
            button.disabled = True
            play.disabled = True
            play.value = play.max
            count_score(vertex_to_expand, True)
    path_x = [segment[0] for segment in current_path]
    path_y = [segment[1] for segment in current_path]
    frontier_x = [segment[0] for segment in frontier]
    frontier_y = [segment[1] for segment in frontier]
    with fig.batch_update():
        fig.data[1].x = path_x
        fig.data[1].y = path_y
        fig.data[2].x = frontier_x
        fig.data[2].y = frontier_y
        fig.data[3].x = [vertex_to_expand[0]]
        fig.data[3].y = [vertex_to_expand[1]]

def create_button():
    button = widgets.Button(description='Чекор')
    button.on_click(update_fig)
    return button
def create_score():
    score = widgets.Text(value='', placeholder='Score')
    return score

def create_play():
    play = widgets.Play(value=0, max=3000, interval=10, show_repeat=False)
    play.observe(update_fig, 'value')
    return play

def create_mountain():
    def f(x, y):
        return 1 - (x - 2 * COLUMNS // 3) ** 2 - (y - ROWS // 2) ** 2

    mountain = [[f(x, y) for x in range(COLUMNS)] for y in range(ROWS)]
    min_value = min(min(mountain))
    for y in range(ROWS):
        for x in range(COLUMNS):
            mountain[y][x] = [mountain[y][x] - min_value] * 3
    
    min_value = min(min(min(mountain)))
    max_value = max(max(max(mountain)))
    mountain_image = go.Image(z=mountain, zmin=[min_value]*4, zmax=[max_value]*4)
    return mountain, mountain_image

def create_fig():
    fig = go.FigureWidget()

    for wall in WALLS:
        fig.add_shape(
            type='rect', fillcolor='RoyalBlue', 
            x0=wall['x'][0], x1=wall['x'][1], 
            y0=wall['y'][0], y1=wall['y'][1])

    fig.add_trace(mountain_image)
    fig.add_scatter(mode='lines', name='Патека')
    fig.add_scatter(mode='markers', name='Фронт')
    fig.add_scatter(marker_symbol='x', marker_size=12, name='Нареден')
    fig.add_scatter(x=[START[0]], y=[START[1]], marker_symbol='x', marker_size=12, name='Почеток')
    for GOAL in GOALS:
        fig.add_scatter(x=[GOAL[0]], y=[GOAL[1]], marker_symbol='x', marker_size=12, name='Цел')

    fig.update_xaxes(range=[-0.5, COLUMNS - 0.5], dtick=1)
    fig.update_yaxes(range=[-0.5, ROWS - 0.5], dtick=1)
    fig.update_layout(width=34*COLUMNS, height=36*ROWS)
    return fig

In [ ]:
ROWS, COLUMNS = 20, 30
count, weight_end = 0, 0
WALLS = [
    {'x': (12.5, 23.5), 'y': (7.5, 9.5)},
    {'x': (8.5, 10.5), 'y': (2.5, 17.5)},
    {'x': (16.5, 19.5), 'y': (12.5, 16.5)},
]

START = (4, 4)
GOALS = [(29, 9), (18, 18)]
MOUNTAIN, mountain_image = create_mountain()
simulation = a_star_search(START, GOALS, alpha=40)

play = create_play()
button = create_button()
text = create_score()
fig = create_fig()
widgets.VBox([widgets.HBox([play, button, text]), fig])

# Izvrsuvanje na site goals

In [31]:
from collections import deque
import heapq
from plotly import graph_objects as go
import ipywidgets as widgets
import math

In [32]:
def breadth_first_search(starting_vertex, goal_vertex):
    """
    Returns the path from starting_vertex to goal_vertex using the breadth first search algorithm.
    """
    visited = {starting_vertex}
    queue = deque([[starting_vertex]])
    while queue:
        vertex_list = queue.popleft()
        vertex_to_expand = vertex_list[-1]
        
        if vertex_to_expand in goal_vertex:
            goal_vertex.remove(vertex_to_expand)
            queue = deque([vertex_list])
            continue
            
        frontier = list(set([q[-1] for q in queue]))
        yield frontier, vertex_list, vertex_to_expand
        
        for neighbour in expand_state(vertex_to_expand):
            if neighbour not in visited:
                visited.add(neighbour)
                queue.append(vertex_list + [neighbour])
    yield [], [], goal_vertex

In [33]:
def depth_first_search(starting_vertex, goal_vertex):
    """
    Returns the path from starting_vertex to goal_vertex using the depth first search algorithm.
    """
    visited = {starting_vertex}
    queue = deque([[starting_vertex]])
    while queue:
        vertex_list = queue.popleft()
        vertex_to_expand = vertex_list[-1]
        
        if vertex_to_expand in goal_vertex:
            goal_vertex.remove(vertex_to_expand)
            queue = deque([vertex_list])
            continue
            
        frontier = list(set([q[-1] for q in queue]))
        yield frontier, vertex_list, vertex_to_expand
        
        for neighbour in expand_state(vertex_to_expand):
            if neighbour not in visited:
                visited.add(neighbour)
                queue.appendleft(vertex_list + [neighbour])
    yield [], [], goal_vertex

In [34]:
def uniform_cost_search(starting_vertex, goal_vertex):
    """
    Returns the path from starting_vertex to goal_vertex using the uniform-cost search algorithm.
    """
    expanded = set()
    queue = [(0, [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        weight, vertex_list = heapq.heappop(queue)
        vertex_to_expand = vertex_list[-1]
        global weight_end
        weight_end = weight
        
        if vertex_to_expand in goal_vertex:
            goal_vertex.remove(vertex_to_expand)
            queue = [((current_a_star_weight, current_path_weight), vertex_list)]
            heapq.heapify(queue)
            continue
        if vertex_to_expand in expanded:
            continue

        frontier = list(set([q[-1][-1] for q in queue]))
        yield frontier, vertex_list, vertex_to_expand
        
        for neighbour in expand_state(vertex_to_expand):
            new_weight = get_weight(vertex_to_expand, neighbour)
            if neighbour not in expanded:
                heapq.heappush(queue, (weight + new_weight, vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)
    yield [], [], goal_vertex

In [35]:
def gready_search(starting_vertex, goal_vertex):
    """
    Returns the path from starting_vertex to goal_vertex using the gready search algorithm.
    """
    expanded = set()
    queue = [(0, [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        weight, vertex_list = heapq.heappop(queue)
        vertex_to_expand = vertex_list[-1]
        global weight_end
        weight_end = weight
        
        if vertex_to_expand in goal_vertex:
            goal_vertex.remove(vertex_to_expand)
            queue = [((current_a_star_weight, current_path_weight), vertex_list)]
            heapq.heapify(queue)
            continue
        
        if vertex_to_expand in expanded:
            continue

        frontier = list(set([q[-1][-1] for q in queue]))
        yield frontier, vertex_list, vertex_to_expand

        for neighbour in expand_state(vertex_to_expand):
            heuristic = min([euclidean_distance(neighbour, temp) for temp in goal_vertex])
            if neighbour not in expanded:
                heapq.heappush(queue, (heuristic, vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)
    yield [], [], goal_vertex

In [36]:
def a_star_search(starting_vertex, goal_vertex, alpha):
    """
    Returns the path from starting_vertex to goal_vertex using the a* search algorithm.
    """
    expanded = set()
    queue = [((0, 0), [starting_vertex])]
    heapq.heapify(queue)
    while queue:
        weight_tupple, vertex_list = heapq.heappop(queue)
        current_a_star_weight, current_path_weight = weight_tupple
        vertex_to_expand = vertex_list[-1]
        global weight_end
        weight_end = current_path_weight
        
        if vertex_to_expand in goal_vertex:
            goal_vertex.remove(vertex_to_expand)
            queue = [((current_a_star_weight, current_path_weight), vertex_list)]
            heapq.heapify(queue)
            continue
            
        if vertex_to_expand in expanded:
            continue

        frontier = list(set([q[-1][-1] for q in queue]))
        yield frontier, vertex_list, vertex_to_expand

        for neighbour in expand_state(vertex_to_expand):
            if neighbour not in expanded:
                new_weight = get_weight(vertex_to_expand, neighbour)
                heuristic = min([euclidean_distance(neighbour, temp) for temp in goal_vertex])
                path_weight = current_path_weight + new_weight
                a_star_weight = path_weight + alpha * heuristic
                heapq.heappush(queue, ((a_star_weight, path_weight), vertex_list + [neighbour]))
        expanded.add(vertex_to_expand)
    yield [], [], goal_vertex

In [37]:
def manhattan_distance(state_1, state_2):
    return abs(state_1[0] - state_2[0]) + abs(state_1[1] - state_2[1])


def euclidean_distance(state_1, state_2):
    return math.sqrt((state_1[0] - state_2[0])**2 + (state_1[1] - state_2[1])**2)


def get_weight(source_state, destination_state):
    source_x, source_y = source_state
    destination_x, destination_y = destination_state
    weight = MOUNTAIN[destination_y][destination_x][0] - MOUNTAIN[source_y][source_x][0]
    return 1 + max(weight, 0)

In [38]:
def count_score(vertex, end=False):
    global count
    text.value = str(count)
    if end:
        text.value = str(count + weight_end)
    count += 1

In [39]:
def expand_state(state):
    next_states = []
    x, y = state

    def is_valid(square):
        x, y = square
        if not(0 <= x < COLUMNS) or not(0 <= y < ROWS):
            return False
        for wall in WALLS:
            if wall['x'][0] < x < wall['x'][1] and wall['y'][0] < y < wall['y'][1]:
                return False
        return True

    neighbour_states = [(x+1, y), (x+1, y+1), (x, y+1), (x-1, y+1), (x-1, y), (x-1, y-1), (x, y-1), (x+1, y-1)]
    for neighbour_state in neighbour_states:
        if is_valid(neighbour_state):
            next_states.append(neighbour_state)

    return next_states

In [40]:
def update_fig(x):
    if button.disabled is True:
        return
    else:
        frontier, current_path, vertex_to_expand = next(simulation)
        count_score(vertex_to_expand)
        if not GOALS:
            button.disabled = True
            play.disabled = True
            play.value = play.max
            count_score(vertex_to_expand, True)
    path_x = [segment[0] for segment in current_path]
    path_y = [segment[1] for segment in current_path]
    frontier_x = [segment[0] for segment in frontier]
    frontier_y = [segment[1] for segment in frontier]
    with fig.batch_update():
        fig.data[1].x = path_x
        fig.data[1].y = path_y
        fig.data[2].x = frontier_x
        fig.data[2].y = frontier_y
        fig.data[3].x = [vertex_to_expand[0]]
        fig.data[3].y = [vertex_to_expand[1]]

def create_button():
    button = widgets.Button(description='Чекор')
    button.on_click(update_fig)
    return button
def create_score():
    score = widgets.Text(value='', placeholder='Score')
    return score

def create_play():
    play = widgets.Play(value=0, max=3000, interval=5, show_repeat=False)
    play.observe(update_fig, 'value')
    return play

def create_mountain():
    def f(x, y):
        return 1 - (x - 2 * COLUMNS // 3) ** 2 - (y - ROWS // 2) ** 2

    mountain = [[f(x, y) for x in range(COLUMNS)] for y in range(ROWS)]
    min_value = min(min(mountain))
    for y in range(ROWS):
        for x in range(COLUMNS):
            mountain[y][x] = [mountain[y][x] - min_value] * 3

    min_value = min(min(min(mountain)))
    max_value = max(max(max(mountain)))
    mountain_image = go.Image(z=mountain, zmin=[min_value]*4, zmax=[max_value]*4)
    return mountain, mountain_image


def create_fig():
    fig = go.FigureWidget()

    for wall in WALLS:
        fig.add_shape(
            type='rect', fillcolor='RoyalBlue', 
            x0=wall['x'][0], x1=wall['x'][1], 
            y0=wall['y'][0], y1=wall['y'][1])

    fig.add_trace(mountain_image)
    fig.add_scatter(mode='lines', name='Патека')
    fig.add_scatter(mode='markers', name='Фронт')
    fig.add_scatter(marker_symbol='x', marker_size=12, name='Нареден')
    fig.add_scatter(x=[START[0]], y=[START[1]], marker_symbol='x', marker_size=12, name='Почеток')
    for index, GOAL in enumerate(GOALS):
        fig.add_scatter(x=[GOAL[0]], y=[GOAL[1]], marker_symbol='x', marker_size=12, name=f"Цел {index+1}")
    fig.update_xaxes(range=[-0.5, COLUMNS - 0.5], dtick=1)
    fig.update_yaxes(range=[-0.5, ROWS - 0.5], dtick=1)
    fig.update_layout(width=34*COLUMNS, height=36*ROWS)
    return fig

In [41]:
ROWS, COLUMNS = 20, 30
count, weight_end = 0, 0
WALLS = [
    {'x': (12.5, 23.5), 'y': (7.5, 9.5)},
    {'x': (8.5, 10.5), 'y': (2.5, 17.5)},
    {'x': (16.5, 19.5), 'y': (12.5, 16.5)},
]

START = (4, 4)
GOALS = [(29, 9), (18, 18), (17, 2)]
MOUNTAIN, mountain_image = create_mountain()
simulation = a_star_search(START, GOALS, alpha=30)

play = create_play()
button = create_button()
text = create_score()
fig = create_fig()
widgets.VBox([widgets.HBox([play, button, text]), fig])

In [42]:
weight_end

365